<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Turku_ERA5_CDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install "cdsapi>=0.7.7"

In [2]:
!pip install netcdf4

In [3]:
import os

# Updated configuration: removed the UID prefix as per the error instructions
content = """url: https://cds.climate.copernicus.eu/api
key: c025f203-5930-4d9c-acd6-699c46be7fd8"""

with open(os.path.expanduser('~/.cdsapirc'), 'w') as f:
    f.write(content)

print("Configuration updated! Now attempting to update the library...")

# Also update your library to the latest version to match the new API
!pip install --upgrade cdsapi

Configuration updated! Now attempting to update the library...


In [4]:
import cdsapi
import os
import calendar

c = cdsapi.Client()

# Range: Full year 2024 + January 2025
tasks = [('2024', str(m).zfill(2)) for m in range(1, 13)] + [('2025', '01')]

print("--- Data Download Started ---")

for i, (year, month) in enumerate(tasks):
    filename = f'turku_{year}_{month}.nc'

    if os.path.exists(filename):
        print(f"[{i+1}/13] {filename} already exists. Skipping.")
        continue

    # Determine days: Full month for 2024, but only Jan 1st for 2025
    if year == '2025':
        days = ['01']
    else:
        last_day = calendar.monthrange(int(year), int(month))[1]
        days = [str(d).zfill(2) for d in range(1, last_day + 1)]

    print(f"[{i+1}/13] Requesting {month}/{year}...")

    try:
        c.retrieve(
            'reanalysis-era5-land',
            {
                'variable': [
                    '2m_temperature',
                    '10m_u_component_of_wind',
                    '10m_v_component_of_wind',
                    'surface_solar_radiation_downwards',
                ],
                'year': year,
                'month': month,
                'day': days,
                'time': [f"{str(h).zfill(2)}:00" for h in range(24)],
                'area': [60.5, 22.5, 60.5, 22.5],
                'format': 'netcdf',
            },
            filename)
        print(f"      ✅ {filename} downloaded.")
    except Exception as e:
        print(f"      ❌ Failed {year}-{month}: {e}")

print("--- All Downloads Complete ---")

2025-12-20 05:30:14,734 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
INFO:ecmwf.datastores.legacy_client:[2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


--- Data Download Started ---
[1/13] Requesting 01/2024...


2025-12-20 05:30:15,217 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2025-12-20 05:30:15,219

8a32d80b5fc7f09d485c3a692c46e6c5.zip:   0%|          | 0.00/108k [00:00<?, ?B/s]

      ✅ turku_2024_01.nc downloaded.
[2/13] Requesting 02/2024...


...

98c4bbae79cb613b059ad3e0653f1a9b.zip:   0%|          | 0.00/108k [00:00<?, ?B/s]

      ✅ turku_2024_12.nc downloaded.
[13/13] Requesting 01/2025...


2025-12-20 05:35:32,960 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2025-12-20 05:35:32,962

5cda661cdca51af284aa26143cb59f6d.zip:   0%|          | 0.00/52.9k [00:00<?, ?B/s]

      ✅ turku_2025_01.nc downloaded.
--- All Downloads Complete ---


In [6]:
import xarray as xr
import pandas as pd
import numpy as np
import glob

# 1. Gather all downloaded NetCDF files
file_list = sorted(glob.glob('turku_*.nc'))
all_data = []

print(f"Starting merge of {len(file_list)} files...")

for f in file_list:
    try:
        # Open dataset (requires 'netcdf4' or 'h5netcdf' engine)
        ds = xr.open_dataset(f)
        # Convert the specific point data to a pandas DataFrame
        df = ds.to_dataframe().reset_index()
        all_data.append(df)
        print(f"✅ Successfully processed: {f}")
    except Exception as e:
        print(f"❌ Error reading {f}: {e}")

if all_data:
    # 2. Concatenate all months into one long time-series
    df_combined = pd.concat(all_data).sort_values('valid_time')

    # 3. Unit Conversions
    # Temperature: Kelvin to Celsius
    df_combined['Air_Temp_C'] = df_combined['t2m'] - 273.15

    # Wind Speed: Calculating magnitude from U and V vectors
    # Formula: sqrt(u^2 + v^2)
    df_combined['Wind_Speed_ms'] = np.sqrt(df_combined['u10']**2 + df_combined['v10']**2)

    # GHI: ERA5-Land 'ssrd' is accumulated Joules/m2 per hour.
    # To get Watts/m2: Divide by 3600 seconds.
    df_combined['GHI_Wm2'] = df_combined['ssrd'] / 3600

    # 4. Final selection and formatting
    # We keep: Timestamp, Temperature, Wind Speed, and Solar Radiation
    final_output = df_combined[['valid_time', 'Air_Temp_C', 'Wind_Speed_ms', 'GHI_Wm2']]
    final_output.columns = ['Timestamp', 'Air_Temp_C', 'Wind_Speed_ms', 'GHI_W_m2']

    # 5. Export to CSV
    output_filename = 'Turku_Hourly_Weather_2024.csv'
    final_output.to_csv(output_filename, index=False)

    print("\n" + "="*30)
    print(f"PROCESSING COMPLETE")
    print(f"File saved: {output_filename}")
    print(f"Total Rows: {len(final_output)}")
    print("="*30)

    # Show the first few lines to verify the data in the GitHub output
    print("\nData Preview:")
    print(final_output.head())
else:
    print("No data frames found. Please check if the .nc files actually downloaded.")

Starting merge of 13 files...
❌ Error reading turku_2024_01.nc: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html
❌ Error reading turku_2024_02.nc: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html
❌ Error reading turku_2024_03.nc: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy']. Consider explicitly selectin